<a href="https://colab.research.google.com/github/andrewbrr4/text-gen/blob/main/text_gen_char.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from google.colab import drive
drive.mount('/content/drive')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/datasets'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Mounted at /content/drive
/content/drive/MyDrive/datasets/david-bowie.txt
/content/drive/MyDrive/datasets/david-bowie-20-3/saved_model.pb
/content/drive/MyDrive/datasets/david-bowie-20-3/keras_metadata.pb
/content/drive/MyDrive/datasets/david-bowie-20-3/variables/variables.index
/content/drive/MyDrive/datasets/david-bowie-20-3/variables/variables.data-00000-of-00001
/content/drive/MyDrive/datasets/david-bowie-word/keras_metadata.pb
/content/drive/MyDrive/datasets/david-bowie-word/saved_model.pb
/content/drive/MyDrive/datasets/david-bowie-word/variables/variables.index
/content/drive/MyDrive/datasets/david-bowie-word/variables/variables.data-00000-of-00001
/content/drive/MyDrive/datasets/david-bowie-20-1/saved_model.pb
/content/drive/MyDrive/datasets/david-bowie-20-1/keras_metadata.pb
/content/drive/MyDrive/datasets/david-bowie-20-1/variables/variables.data-00000-of-00001
/content/drive/MyDrive/datasets/david-bowie-20-1/variables/variables.index
/content/drive/MyDrive/datasets/david-bow

In [ ]:
import tensorflow as tf
import keras
import re
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, GRU, LSTM, SimpleRNN, Dense, Dropout
from keras.preprocessing.sequence import pad_sequences

# Define the functions to create a model based off an artists lyrics and generate new lyrics from that model

In [ ]:
chars = list('abcdefghijklmnopqrstuvwxyz')
chars.append(' ')
chars.append('\n')
char_to_index = dict((c, i) for i, c in enumerate(chars))
index_to_char = dict((i, c) for i, c in enumerate(chars))

def create_char_model(artist,char_window,step,num_epochs):
    
    with open('/content/drive/MyDrive/datasets/'+artist+'.txt') as f:
        text = f.read().lower()

    text = re.sub('[^a-z \n]','',text)

    inputs = []
    next_chars = []

    for i in range(0,len(text)-char_window,step):
        inputs.append(text[i:i+char_window])
        next_chars.append(text[i+char_window])

    pairs = zip(inputs,next_chars)
    list(pairs)[:100]

    X = np.zeros((len(inputs),char_window),dtype=np.float32)

    for i in range(len(inputs)):
        for j in range(char_window):
            X[i,j] = char_to_index[inputs[i][j]]

    y = np.zeros(len(next_chars),dtype=np.float32)

    for i in range(len(next_chars)):
        y[i] = char_to_index[next_chars[i]]

    y = to_categorical(y)
    
    model = Sequential()

    model.add(Embedding(28,64,input_length=char_window))
    model.add(LSTM(256,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(256))
    model.add(Dropout(0.2))
    model.add(Dense(28,activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    # model.fit(X,y,epochs=5,batch_size=20)
    model.fit(X,y,epochs=num_epochs,batch_size=20)
    model.save('/content/drive/MyDrive/datasets/'+artist+'-'+str(char_window)+'-'+str(step))
    return model

In [ ]:
def softmax(z):
    return np.exp(z)/sum(np.exp(z))

def temp_scale(conditional_probability, temperature=1.0):
    conditional_probability = np.asarray(conditional_probability).astype("float64")
    conditional_probability = np.log(conditional_probability) / temperature
    reweighted_conditional_probability = softmax(conditional_probability)
    probas = np.random.multinomial(1, reweighted_conditional_probability, 1)
    return np.argmax(probas)

def generate_line(model,context,temp,char_window):
    line = context
    next_char = ''
    counter = 0
    while counter<100:
        X = [[char_to_index[line[i]] for i in range(0,len(line))]]
        X = pad_sequences(X,maxlen=char_window)
        y = model.predict(X)
        next_char = index_to_char[temp_scale(y[0,:],temp)]
        counter+=1
        line+=next_char
    return line

# Now lets have some fun emulating different artists!!

## char = 20, step = 3

In [ ]:
char_window = 20
step = 3
epochs = 6
temps = [.01, 0.2, 0.5, 1.0, 1.2, 1.5]

In [ ]:
bowie_bot = create_char_model('david-bowie',char_window,step,epochs)

Epoch 1/6
16978/16978 [==============================] - 2511s 148ms/step - loss: 1.7894 - accuracy: 0.4556
Epoch 2/6
16978/16978 [==============================] - 2451s 144ms/step - loss: 1.4417 - accuracy: 0.5549
Epoch 3/6
16978/16978 [==============================] - 2511s 148ms/step - loss: 1.3494 - accuracy: 0.5827
Epoch 4/6
16978/16978 [==============================] - 2539s 150ms/step - loss: 1.3031 - accuracy: 0.5952
Epoch 5/6
16978/16978 [==============================] - 2555s 150ms/step - loss: 1.2801 - accuracy: 0.6024
Epoch 6/6
16978/16978 [==============================] - 2506s 148ms/step - loss: 1.2716 - accuracy: 0.6048


INFO:tensorflow:Assets written to: /content/drive/MyDrive/datasets/david-bowie-20-3/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/datasets/david-bowie-20-3/assets


In [ ]:
bowie_bot = keras.models.load_model('/content/drive/MyDrive/datasets/david-bowie-'+str(char_window)+'-'+str(step))

In [ ]:
output = '********** WINDOW = ' + str(char_window) + ' STEP = ' + str(step) + ' ***********\n'

for temp in temps:
    output += '**** TEMP = ' + str(temp) + ' ****\n'
    output += re.sub(
      r'[\n]+', '\n',generate_line(bowie_bot,'i want to',temp,char_window)
    )
    output += '\n\n'

print(output)
file = open("/content/drive/MyDrive/datasets/lyrics.txt", "a")  # append mode
file.write(output+'\n\n')
file.close()

********** WINDOW = 20 STEP = 3 ***********
**** TEMP = 0.01 ****
i want to be free
i was all the world can is a care
i was alright
the street where i can do


**** TEMP = 0.2 ****
i want to be free
i was alright
the world can is a crazy day
the street where i can do
i cant trace time to

**** TEMP = 0.5 ****
i want to be free
and the arms in the days
the wall like a shadows
im not all the street naked passon
the

**** TEMP = 1.0 ****
i want to be free
keep so many ghinging
if it ill i live in
so we cant give around control to major tom
turn 

**** TEMP = 1.2 ****
i want to live
wetrendt you
stepsing her seens my headso
red come the gunner
tellgring a phollax dream
you wa

**** TEMP = 1.5 ****
i want to make the real melcha
no contracocf protect things
yosr
jlves he never the drystrangung or soandwife




In [ ]:
output = '********** WINDOW = ' + str(char_window) + ' STEP = ' + str(step) + ' ***********\n'

for temp in temps:
    output += '**** TEMP = ' + str(temp) + ' ****\n'
    output += re.sub(
      r'[\n]+', '\n',generate_line(bowie_bot,'today',temp,char_window)
    )
    output += '\n\n'

print(output)
file = open("/content/drive/MyDrive/datasets/lyrics.txt", "a")  # append mode
file.write(output+'\n\n')
file.close()

********** WINDOW = 20 STEP = 3 ***********
**** TEMP = 0.01 ****
today the street where its too late to be babe


**** TEMP = 0.2 ****
today the street where i can do
baby baby baby baby baby baby
baby baby baby baby baby


**** TEMP = 0.5 ****
today they say
they say jump
but the babys a shall want to be a sailor white noise of sun
paddy whe

**** TEMP = 1.0 ****
today living  its the waet the aagic or as dont yeah
kyere i cant get up not somebody sucked the waters o

**** TEMP = 1.2 ****
todays once ence
freay insidos
ex youre now  loxhoys fate looking stails grast in biggle cabors 
well 

**** TEMP = 1.5 ****
today
thank other baby
cover up
knowledgh diggingsy callio
its jutter alone
ju im not aloan
ak reflama




In [ ]:
output = '********** WINDOW = ' + str(char_window) + ' STEP = ' + str(step) + ' ***********\n'

for temp in temps:
    output += '**** TEMP = ' + str(temp) + ' ****\n'
    output += re.sub(
      r'[\n]+', '\n',generate_line(bowie_bot,'what am i',temp,char_window)
    )
    output += '\n\n'

print(output)
file = open("/content/drive/MyDrive/datasets/lyrics.txt", "a")  # append mode
file.write(output+'\n\n')
file.close()

********** WINDOW = 20 STEP = 3 ***********
**** TEMP = 0.01 ****
what am i fly down
i was alright
the street where i can do


**** TEMP = 0.2 ****
what am i fan make me all the world
im afraid i cant
im afraid of americans
im afraid of americans
im afraid 

**** TEMP = 0.5 ****
what am i fan
im a back was anywity insane
i said a dog a change and the rage the street and really got a m

**** TEMP = 1.0 ****
what am i happame
to be on the rage all the that all is holytgithing to space
my way to suck out for you
so 

**** TEMP = 1.2 ****
what am inorakponia night
but i nust could ellah ooh ah yeah
im
dont get it
nothing and papertion andthgess 

**** TEMP = 1.5 ****
what am i
funkathing is a cone
and me
johnnys in american
im afraid i cant help it
if i want it for togetthod




## char = 20, step = 1

In [ ]:
char_window = 20
step = 1
epochs = 3
temps = [.01, 0.2, 0.5, 1.0, 1.2, 1.5]

In [ ]:
bowie_bot = create_char_model('david-bowie',char_window,step,epochs)

Epoch 1/3
50932/50932 [==============================] - 6197s 122ms/step - loss: 1.5546 - accuracy: 0.5229
Epoch 2/3
50932/50932 [==============================] - 6185s 121ms/step - loss: 1.3318 - accuracy: 0.5874
Epoch 3/3
50932/50932 [==============================] - 6207s 122ms/step - loss: 1.3084 - accuracy: 0.5938


INFO:tensorflow:Assets written to: /content/drive/MyDrive/datasets/david-bowie-20-1/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/datasets/david-bowie-20-1/assets


In [ ]:
bowie_bot = keras.models.load_model('/content/drive/MyDrive/datasets/david-bowie-'+str(char_window)+'-'+str(step))

In [ ]:
output = '********** WINDOW = ' + str(char_window) + ' STEP = ' + str(step) + ' ***********\n'

for temp in temps:
    output += '**** TEMP = ' + str(temp) + ' ****\n'
    output += re.sub(
      r'[\n]+', '\n',generate_line(bowie_bot,'i want to',temp,char_window)
    )
    output += '\n\n'

print(output)
file = open("/content/drive/MyDrive/datasets/lyrics.txt", "a")  # append mode
file.write(output+'\n\n')
file.close()

********** WINDOW = 20 STEP = 1 ***********
**** TEMP = 0.01 ****
i want to live my life
oh yeah


**** TEMP = 0.2 ****
i want to live my life
and the sun is a star
so i cant help thinking about me
something in ou

**** TEMP = 0.5 ****
i want to say or do
and theres nothing were just for me
the stars of leathing the will they let me down
a

**** TEMP = 1.0 ****
i want to live
freak on tonight
thats all well
i like it
we can be neath hunk now 
really le dont be away
ha

**** TEMP = 1.2 ****
i want to the heat walking me along
swove cie telling my new
and right angels thun their days my from me
wat

**** TEMP = 1.5 ****
i want to live
sun me close
emerdicains beyperiorniuf
now juime
neplen drie wonder
i
have everyone we explin




In [ ]:
output = '********** WINDOW = ' + str(char_window) + ' STEP = ' + str(step) + ' ***********\n'

for temp in temps:
    output += '**** TEMP = ' + str(temp) + ' ****\n'
    output += re.sub(
      r'[\n]+', '\n',generate_line(bowie_bot,'today',temp,char_window)
    )
    output += '\n\n'

print(output)
file = open("/content/drive/MyDrive/datasets/lyrics.txt", "a")  # append mode
file.write(output+'\n\n')
file.close()

********** WINDOW = 20 STEP = 1 ***********
**** TEMP = 0.01 ****
today i cant help thinking about me


**** TEMP = 0.2 ****
today i cant help thinking about me
something in our back
the sun machine is a better way


**** TEMP = 0.5 ****
today yeah
get your arms the sun
know it like a good time i called my star
they wore if you made your pre

**** TEMP = 1.0 ****
today until you silly
hey man he tanged the pice
alfith real cool world
im afraid i cant help it
im afr

**** TEMP = 1.2 ****
today
but look out free
ooh ooh
if
im derigetee
no hellbill i could run my brems end has goine
yeah
ah 

**** TEMP = 1.5 ****
today can you zeel snades
and out a saddaspendy hard
ev waiting forgened behullet
huplion
he do
happyma




In [ ]:
output = '********** WINDOW = ' + str(char_window) + ' STEP = ' + str(step) + ' ***********\n'

for temp in temps:
    output += '**** TEMP = ' + str(temp) + ' ****\n'
    output += re.sub(
      r'[\n]+', '\n',generate_line(bowie_bot,'what am i',temp,char_window)
    )
    output += '\n\n'

print(output)
file = open("/content/drive/MyDrive/datasets/lyrics.txt", "a")  # append mode
file.write(output+'\n\n')
file.close()

********** WINDOW = 20 STEP = 1 ***********
**** TEMP = 0.01 ****
what am is a couple
and the sun machine is coming down


**** TEMP = 0.2 ****
what am i can see her face is a mess
rebel rebel your face is a mess
and the days and she was alr

**** TEMP = 0.5 ****
what am is sailory little wonder you
its the heart and i love
and they take us to be been so long so i long

**** TEMP = 1.0 ****
what am is see faping man
hold helping home
for one dayss seven
every much to be my good
i dont
dont let me b

**** TEMP = 1.2 ****
what am is only for the stars chectred dridhere first tiny a ultiel
sevenad memy
hey
says alone we called my 

**** TEMP = 1.5 ****
what am im god is bodes
lhoys go kind ah heve
couldnt lives wonder heaven
chem wo 
kaindh lhh can he he lwon




## char = 100, step = 5

In [ ]:
char_window = 50
step = 3
epochs = 5
temps = [.01, 0.2, 0.5, 1.0, 1.2, 1.5]

In [ ]:
bowie_bot = create_char_model('david-bowie',char_window,step,epochs)

Epoch 1/5
16977/16977 [==============================] - 5408s 318ms/step - loss: 1.7867 - accuracy: 0.4561
Epoch 2/5
16977/16977 [==============================] - 5432s 320ms/step - loss: 1.4405 - accuracy: 0.5547
Epoch 3/5
16977/16977 [==============================] - 5717s 337ms/step - loss: 1.3463 - accuracy: 0.5822
Epoch 4/5
16977/16977 [==============================] - 5602s 330ms/step - loss: 1.3010 - accuracy: 0.5965
Epoch 5/5
16977/16977 [==============================] - 5502s 324ms/step - loss: 1.2732 - accuracy: 0.6040


INFO:tensorflow:Assets written to: /content/drive/MyDrive/datasets/david-bowie-50-3/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/datasets/david-bowie-50-3/assets


In [ ]:
bowie_bot = keras.models.load_model('/content/drive/MyDrive/datasets/david-bowie-'+str(char_window)+'-'+str(step))

In [ ]:
output = '********** WINDOW = ' + str(char_window) + ' STEP = ' + str(step) + ' ***********\n'

for temp in temps:
    output += '**** TEMP = ' + str(temp) + ' ****\n'
    output += re.sub(
      r'[\n]+', '\n',generate_line(bowie_bot,'i want to',temp,char_window)
    )
    output += '\n\n'

print(output)
file = open("/content/drive/MyDrive/datasets/lyrics.txt", "a")  # append mode
file.write(output+'\n\n')
file.close()

********** WINDOW = 50 STEP = 3 ***********
**** TEMP = 0.01 ****
i want to say


**** TEMP = 0.2 ****
i want to sear the street


**** TEMP = 0.5 ****
i want to say a the same of the same
in the sky and the moon
while the caled the same to such a better full


**** TEMP = 1.0 ****
i want to face
right live  
yary on your mindlit day from no money you
and im fly red bocent perchly big boy

**** TEMP = 1.2 ****
i want to send life your trast
byebye love love
bse of polister repuice
that i put with him
youve nabe bass a

**** TEMP = 1.5 ****
i want togaze little shany
sendee
brobrow down cbing
his lwvails
saver
sfinged i stora befemby glamp and and




In [ ]:
output = '********** WINDOW = ' + str(char_window) + ' STEP = ' + str(step) + ' ***********\n'

for temp in temps:
    output += '**** TEMP = ' + str(temp) + ' ****\n'
    output += re.sub(
      r'[\n]+', '\n',generate_line(bowie_bot,'today',temp,char_window)
    )
    output += '\n\n'

print(output)
file = open("/content/drive/MyDrive/datasets/lyrics.txt", "a")  # append mode
file.write(output+'\n\n')
file.close()

********** WINDOW = 50 STEP = 3 ***********
**** TEMP = 0.01 ****
today int mart of the street
i cant help thinking about me


**** TEMP = 0.2 ****
today and i can see me to stay


**** TEMP = 0.5 ****
today in the strange throw with men and the sky
she says the way that you cant say no do
so st

**** TEMP = 1.0 ****
today looks
mama amaha that my crazy icle wutchery bowlead
thesn mim time time man
choo dig bar re da da 

**** TEMP = 1.2 ****
today if hope sound burrhise nhortisre eyes
rebel 
now my moonbiet 
grow hum here is no one doesnnt fo

**** TEMP = 1.5 ****
today we talk trmach yourselvey over
you fall that brain
breathil 
it
lear wime
uheathery free bite
iffer




In [ ]:
output = '********** WINDOW = ' + str(char_window) + ' STEP = ' + str(step) + ' ***********\n'

for temp in temps:
    output += '**** TEMP = ' + str(temp) + ' ****\n'
    output += re.sub(
      r'[\n]+', '\n',generate_line(bowie_bot,'what am i',temp,char_window)
    )
    output += '\n\n'

print(output)
file = open("/content/drive/MyDrive/datasets/lyrics.txt", "a")  # append mode
file.write(output+'\n\n')
file.close()

********** WINDOW = 50 STEP = 3 ***********
**** TEMP = 0.01 ****
what am i cant help thinking about me


**** TEMP = 0.2 ****
what am i love you the street of the stars
and the sun machine is coming to say
i cant help thinking about 

**** TEMP = 0.5 ****
what am i cant stay and the pail cant see me
the sun machine is coming to such a shook the street
now now i

**** TEMP = 1.0 ****
what am in the poat man sausend
when hey man so saim on suicide
im a bread of me of pourent your living rose


**** TEMP = 1.2 ****
what am i will gove you stoppeajir crowns of pirarria
i want tall tell your mouth
til you like a better ba

**** TEMP = 1.5 ****
what am in our faced reweals
sarls manp
cruight mide and eatters are all unem to
kissel sava thigh wenky sky



